<h1 div class='alert alert-success'><center> TPS-Set: Feature Engineering</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  1. IMPORTAÇÕES </div>

## 1.1. Bibliotecas 

In [ ]:
import warnings
import random
import os
import gc
import shap
import torch

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import scikitplot        as skplt # conda install -c conda-forge/label/cf201901 scikit-plot

In [ ]:
from sklearn.metrics           import silhouette_samples, silhouette_score
from sklearn.model_selection   import train_test_split, KFold #, RepeatedStratifiedKFold, StratifiedKFold  
from sklearn                   import metrics
from sklearn.preprocessing     import QuantileTransformer, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.cluster           import KMeans

In [ ]:
from yellowbrick.cluster       import KElbowVisualizer, SilhouetteVisualizer
#from prettytable               import PrettyTable
#from sklearn.ensemble          import IsolationForest
from datetime                  import datetime

In [ ]:
import xgboost                 as xgb

## 1.2. Funções

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    
    #pd.set_option('display.max_rows', 5)
    #pd.set_option('display.max_columns', 500)
    #pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.rcParams['font.size'] = 12
    plt.title('Precision Recall vs threshold')
    plt.xlabel('Threshold')
    plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls[:-1], precisions[:-1], "b-", label="Precision")
    
    plt.rcParams['font.size'] = 12
    plt.title('Precision vs recall')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('XGBR ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def confusion_plot(matrix, labels = None, title = None):
    
    """ Exibir matriz de confusão binária como um mapa de calor Seaborn """
    plt.figure(figsize=(7,5))
    
    labels = labels if labels else ['Negative (0)', 'Positive (1)']
    
    #labels      = ['No Churn', 'Churn']
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    sns.heatmap(data        = matrix, 
                cmap        = 'Blues', 
                annot       = True, 
                fmt         = 'd',
                xticklabels = labels, 
                yticklabels = labels, 
                ax          = ax)
    
    ax.set_xlabel('\n PREVISTO', fontsize=15)
    ax.set_ylabel('REAL \n', fontsize=15)
    ax.set_title(title)
    
    plt.close()
    
    return fig;

In [ ]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def diff(t_a, t_b):
    from dateutil.relativedelta import relativedelta
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [ ]:
!mkdir img
!mkdir Data
!mkdir Data/pkl
!mkdir Data/submission

!mkdir model
!mkdir model/preds
!mkdir model/optuna

!mkdir model/preds/test
!mkdir model/preds/test/n1
!mkdir model/preds/test/n2
!mkdir model/preds/test/n3

!mkdir model/preds/train
!mkdir model/preds/train/n1
!mkdir model/preds/train/n2
!mkdir model/preds/train/n3
!mkdir model/preds/param

## 1.3. Carregar Dados

In [ ]:
path   = '../input/tps11001/'
target = 'target'

In [ ]:
%%time
df1_train     = jb.load(path + 'df1_nb_01_train.pkl.z')
df1_test      = jb.load(path + 'df1_nb_01_test.pkl.z')
df_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

df1_train.drop('id', axis=1, inplace=True)
df1_test.drop('id', axis=1, inplace=True)

gc.collect()

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_train.head()

# <div class="alert alert-success"> 2. Feature Engineering </div>

In [ ]:
feature_num = df1_train.columns.to_list() 
feature_num.remove('target')

## 2.1. Step 01
Vamos fazer uma clusterização e gerar uma nova variável, isso pode ajudar na identificação de novos padrões e melhor a predição dos algoritmos, caso não surta efeito podemos remover a variável.    

In [ ]:
%%time
scaler        = QuantileTransformer(output_distribution='normal', random_state=0)
df1_train_qt  = df1_train.copy().drop('target', axis=1) 
df1_train_qt  = pd.DataFrame(scaler.fit_transform(df1_train_qt), columns=feature_num)
df1_test_qt   = pd.DataFrame(scaler.fit_transform(df1_test), columns=feature_num)

In [ ]:
%%time 

plt.figure(figsize=(12, 7))
visualizer_1 = KElbowVisualizer(KMeans(random_state=12359), k=(2,12))
visualizer_1.fit(df1_train_qt.head(300000));
visualizer_1.poof();

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>

De acordo com o gráfico acima, podemos criar 5 clusteres nos datasets
    
    
</div>


In [ ]:
model_kmeans = KMeans(n_clusters=5, random_state=59)
model_kmeans.fit(df1_train_qt);

clusters_train = model_kmeans.predict(df1_train_qt)
clusters_test  = model_kmeans.predict(df1_test_qt)

df1_train['fe_cluster'] = clusters_train
df1_test['fe_cluster']  = clusters_test

df1_train.shape, df1_test.shape

In [ ]:
df1_train.head()

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>
Vamos transforma a cluster em dammy.
    
    
</div>


In [ ]:
df1_train = pd.get_dummies(df1_train, columns=['fe_cluster'])
df1_test = pd.get_dummies(df1_test, columns=['fe_cluster'])

df1_train.shape, df1_test.shape

In [ ]:
df1_train.head()

In [ ]:
del df1_test_qt, df1_train_qt

## 2.2. Step 02

In [ ]:
df1_train['fe_mean']        = df1_train[feature_num].mean(axis=1)
df1_test['fe_mean']         = df1_test[feature_num].mean(axis=1)
df1_train['fe_median']      = df1_train[feature_num].median(axis=1)
df1_test['fe_median']       = df1_test[feature_num].median(axis=1)
df1_train['fe_min']         = df1_train[feature_num].min(axis=1)
df1_test['fe_min']          = df1_test[feature_num].min(axis=1)
df1_train['fe_max']         = df1_train[feature_num].max(axis=1)
df1_test['fe_max']          = df1_test[feature_num].max(axis=1)
df1_train['fe_skew']        = df1_train[feature_num].skew(axis=1)
df1_test['fe_skew']         = df1_test[feature_num].skew(axis=1)

gc.collect()

df1_train.shape, df1_test.shape

In [ ]:
df1_train.head()

Vamos salvar os dataset, caso seja necessário refazer o processo podemos partir desse ponto. 

In [ ]:
jb.dump(df1_train,  "Data/pkl/df2_nb_02_train.pkl.z")
jb.dump(df1_test,  "Data/pkl/df2_nb_02_test.pkl.z")

del df1_train, df1_test
gc.collect()

# <div class="alert alert-success"> 3. Split Train/Test </div>

In [ ]:
df2_train = jb.load('Data/pkl/df2_nb_02_train.pkl.z')
df2_test  = jb.load('Data/pkl/df2_nb_02_test.pkl.z')

df2_train.shape, df2_test.shape

In [ ]:
print(feature_num)

In [ ]:
X      = df2_train.drop(['target'], axis=1)
y      = df2_train['target']
#cols   = X.columns
X_test = df2_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 0)

del df2_train, df2_test
gc.collect()

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape 

In [ ]:
X_test.head()

# <div class="alert alert-success"> 4. Modelagem </div>

In [ ]:
seed   = 12359
params = {'objective'     : 'binary:logistic',    
          'eval_metric'   : 'auc',
          'random_state'  : seed}

if torch.cuda.is_available():           
    params.update({'predictor'  : 'gpu_predictor', 
                   'tree_method': 'gpu_hist', 
                   'gpu_id'     :  0})

params

## 4.1. Validação Cruzada

In [ ]:
path=''

def save_data_model(model_, model_name_, path_, X_train_prob_, y_hat_test_, score_, seed_, level_='1'):

    level_ = 'n'+ level_ + '/'

    if score_>.6:          

        path_name_param = path_ + 'model/preds/param/' + model_name_.format(score_, seed_)
        path_name_train = path_ + 'model/preds/train/' + level_ + model_name_.format(score_, seed_)
        path_name_test  = path_ + 'model/preds/test/'  + level_ + model_name_.format(score_, seed_)    
        path_name_model = path_ + 'model/mdl/'         + model_name_.format(score_, seed_)    

        jb.dump(X_train_prob_, path_name_train)
        jb.dump(y_hat_test_, path_name_test)
        jb.dump(model_, path_name_model)
        jb.dump(pd.DataFrame([model_.get_params()]), path_name_param)   

        if score_>.65:
            # Gerar o arquivo de submissão 
            df_submission['target'] = y_hat_test_
            name_file_sub =  path_ + 'Data/submission/tunning/' + model_name_.format(score_, seed_) + '.csv'
            df_submission.to_csv(name_file_sub, index = False)

def cross_val_model(model, X_train_, y_train_, X_test_,  scalers, name_model, 
                    FOLDS=5, verbose=False, seed=12359, use_ntree_limit=False): 
    
    time_s = datetime.now()
        
    mdl_train   = []
    feature_imp = 0 
    auc_best    = 0
    
    for scaler in scalers: 
        
        gc.collect()

        df_submission.claim = 0           
        feature_imp_best    = 0   
        X_train_prob        = []
        auc                 = []
        lloss               = []
        f1                  = []
        ntree               = []
        n_estimators        = model.get_params()['n_estimators'] 
        kfold               = KFold(n_splits=FOLDS, random_state=seed, shuffle=True)

        if scaler!=None:
            X_ts = scaler.fit_transform(X_test_.copy())
        else:
            X_ts = X_test_.copy()

        print('='*80)
        print('Scaler: {} - n_estimators: {}'.format(scaler,n_estimators))
        print('='*80)

        for i, (train_idx, test_idx) in enumerate(kfold.split(X_train_)):

            time_start = datetime.now()
            
            i+=1

            X_tr, y_tr = X_train_.iloc[train_idx], y_train_.iloc[train_idx]
            X_vl, y_vl = X_train_.iloc[test_idx], y_train_.iloc[test_idx]

            # Scaler
            if scaler!=None:    
                X_tr = scaler.fit_transform(X_tr)
                X_vl = scaler.fit_transform(X_vl)                

            model.fit(X_tr, y_tr, 
                      eval_set              = [(X_tr,y_tr), (X_vl,y_vl)],
                      early_stopping_rounds = int(n_estimators*.1), 
                      verbose               = verbose
                     )
            
            if use_ntree_limit:
                y_hat_prob  = model.predict_proba(X_vl, ntree_limit=model.best_ntree_limit)[:, 1] # 
                best_ntree_ = model.best_ntree_limit
            else: 
                y_hat_prob  = model.predict_proba(X_vl)[:, 1] # 
                best_ntree_ = n_estimators
                            
            y_hat         = (y_hat_prob >.5).astype(int) 
            log_loss_     = metrics.log_loss(y_vl, y_hat_prob)                
            f1_score_     = metrics.f1_score(y_vl, y_hat)                    
            auc_          = metrics.roc_auc_score(y_vl, y_hat_prob)

            elapsed  = diff(time_start, datetime.now())
            
            stop = '*' if n_estimators > best_ntree_ else ' '
            msg  = '[Fold {}] AUC: {:.5f} - F1: {:.5f} - L. LOSS: {:.5f} {} {} {}'
            print(msg.format(i, auc_, f1_score_,log_loss_, stop, best_ntree_, elapsed))

            # Getting mean feature importances (i.e. devided by number of splits)
            feature_imp  += model.feature_importances_ / FOLDS
            
            df_submission['target'] += model.predict_proba(X_ts)[:, 1] / FOLDS
            
                  
            f1.append(f1_score_)
            lloss.append(log_loss_)
            auc.append(auc_)
            ntree.append(best_ntree_)
            
            X_train_prob.append(auc_)  
            
            gc.collect()
                        
        auc_mean   = np.mean(auc)
        auc_std    = np.std(auc)
        lloss_mean = np.mean(lloss)
        f1_mean    = np.mean(f1)
        ntree_mean = np.mean(ntree)
        
        if auc_mean > auc_best: 
            auc_best          = auc_mean
            f1_best           = f1_mean
            lloss_best        = lloss_mean
            model_best        = model
            feature_imp_best  = feature_imp
            scaler_best       = scaler
           
        elapsed  = diff(time_s, datetime.now())
            
        print('-'*80)
        msg = '[Mean Fold] AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. LOSS: {:.5f} - {} {}'
        print(msg.format(auc_mean,auc_std, f1_mean, lloss_mean, ntree_mean, elapsed))
        print('='*80)
        print('')

        # Gerar o arquivo de submissão 
        name_file_sub = 'Data/submission/' + name_model + '_' + str(scaler).lower()[:4] + '.csv'
        df_submission.to_csv(name_file_sub.format(auc_mean), index = False)

        gc.collect()
     
    mdl_name_best = 'model/' + name_model.format(auc_mean)
    
    jb.dump(model_best, mdl_name_best)
    random       = str(np.random.rand(1)[0]).replace('.','')
    #model_name_  = name_model + '_score_{:2.5f}_{}.pkl.z'#.format(auc_best, random)
    
    # save_data_model(model_        = model, 
    #                 model_name_   = model_name_, 
    #                 path_         = path, 
    #                 X_train_prob_ = X_train_prob, 
    #                 y_hat_test_   = df_submission['target'], 
    #                 score_        = auc_best, 
    #                 seed_         = seed, 
    #                 level_        = '1')
    
    print()
    print('='*80)
    print('Scaler Best: {}'.format(scaler_best))
    print('AUC        : {:2.5f}'.format(auc_best))
    print('F1-Score   : {:2.5f}'.format(f1_best))
    print('L. Loss    : {:2.5f}'.format(lloss_best))
    print('='*80)
    print()
            
    gc.collect()  
    
    return model_best

In [ ]:
%%time

params.update({'n_estimators': 1000})

scalers = [None, 
           StandardScaler(), 
           RobustScaler(),           
           QuantileTransformer(output_distribution='normal', random_state=0)]

model_best = cross_val_model(model           = xgb.XGBClassifier(**params), 
                             X_train_        = X, 
                             y_train_        = y,
                             X_test_         = X_test,                                            
                             scalers         = scalers, 
                             name_model      = 'xgb_005_fe_{:2.5f}', 
                             FOLDS           = 5, 
                             seed            = seed, 
                             use_ntree_limit = True
                             ) 

gc.collect()
print()

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>

Como podemos obsevar acima, a média de AUC na validação cruzada foi de 0.73142 e nas submissões do kaggle foram os seguintes scores:

- **`None`**: No treino obtever AUC  de 0.73179 e no kable AUC de 0.73818; <br>
- **`RobustScaler`**: No treino obtever AUC  de 0.73139 e no kable AUC de 0.73534; <br>
- **`QuatileTransfomer`**: No treino obtever AUC 0.73048 e no kable AUC de 0.73531 <br> 

    
</div>

### 4.2.4.1. Feature Import Modelo

In [ ]:
feat_imp_best    = model_best.feature_importances_
feature_imp_     = feat_imp_best
df               = pd.DataFrame()
df["Feature"]    = X.columns
df["Importance"] = feature_imp_ / feature_imp_.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 70))
bars    = ax.barh(df["Feature"], 
                  df["Importance"], 
                  height    = 0.4,
                  color     = "mediumorchid", 
                  edgecolor = "black")

ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
#ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adicionando rótulos na parte superior
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverter a direção do eixo y 
plt.gca().invert_yaxis()

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>

Acima podemos observar que algumas das variáveis que criamos estão entre as 25 variáveis mais importantes para o modelo XGB, porém não gosto muito desse jeito de analisar as variáveis mais importantes, vamos utilizar o `Shap` que utilza vários métodos diferentes para encontrar as variáveis importantes. 
    
SHAP (SHapley Additive exPlanations) é uma abordagem teórica de jogos para explicar a saída de qualquer modelo de aprendizado de máquina, ver o [artigos](https://github.com/slundberg/shap#citations) para detalhes e citações.
    
</div>

### 4.2.4.1. SHAP Values
Explicar as previsões do modelo usando SHAP.

In [ ]:
explainer = shap.Explainer(model_best)
shap_values = explainer(X)

In [ ]:
shap.plots.waterfall(shap_values[0],max_display=110)

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>
O gráfico acima mostra as variáveis e suas contribuições na predição do modelo, as variáveis que empurram a previsão para cima são mostradas em vermelho, e as que empurram o previsão para baixo são mostradas em azul.<br>
As variáveis que criamos que empurram a previsão para baixo são: e_mean, fe_max, fe_skew, vamos visualzar a mesma explicação utilizando o gráfico de força. 
</div>

In [ ]:
shap.plots.force(shap_values[0])

<div class="alert alert-info" role="alert"> 
    
Vamos dar uma olhda no resumo das das variáveis, o gráfico abaixo classifica os recursos pela soma das magnitudes dos valores de SHAP em todas as amostras e usa os valores de SHAP para mostrar a distribuição dos impactos que cada variável tem na predição do modelo. A cor representa o valor da variável (vermelho alto, azul baixo). 
    
</div>

In [ ]:
shap.plots.beeswarm(shap_values)

<div class="alert alert-info" role="alert">    
**`NOTA:`** <br>
O gráfico acima revela que as variáveis f34, f43 e f8 tem alto impacto na previsão do XGB.      
</div>

In [ ]:
shap.plots.beeswarm(shap_values, max_display=111)

<div class="alert alert-info" role="alert">
    
Também podemos apenas pegar o valor absoluto médio dos valores de SHAP para cada variável para obter um gráfico de barra padrão:
</div>

In [ ]:
shap.plots.bar(shap_values, max_display=111)

# <div class="alert alert-success"> 5. Conclusão </div>

<div class="alert alert-info" role="alert">    
Neste notebook criamos novas variáveis utilizando a clusterização e variáveis estatísticas, com a finalidade de ajudar os modelos a identificar padrões no dados para melhor as previsões, na validação realizada com XGB obtivemos uma melhora, porém nas submissões não tivemos melhora. <br>
    
Além da criação de novas variávies, utilizamos o SHAP Values para avaliar o impacto das novas variáveis e antigas nas previsões do modelo XGB, podemos destacar que as variáveis: **fe_mean, fe_max, fe_skew** que foram criadas tem impacto negativo nas predições, assim como outras variáveis puxam a predição para baixo, esse talvez seja o favor de não temos melhoria nas nossas submissões, sendo assim, são variáveis contidas a serem removidas do modelo. <br>
    
Um ponto importante que devemos destacar é que ainda não fizemos os ajuste de parametros e estamos trabalhando apenas com um predito (XGB), sendo assim, essas variáveis podem ter valor preditivos relevantes para outros classificadores, os quais vamos teste em outros notebooks.
    
Nos próximos notebooks vamos fazer o ajuste de parametros para os seguintes classificadores: 
- XGB
- LGBM; 
- CatBoost; 
- Rede Neural; 
- KNN;
- SVN; 
    
A ideia é criar uma stacking, sendo assim, vamos salvar todos os modelos.
    
</div>